In [1]:
import os

from sleep_events import connect_to_firebase

firebase = connect_to_firebase()

In [2]:
import pandas as pd

docs = firebase.collection('notesExperimental').stream()

note_records = [{**doc.to_dict(), 'id': doc.id} for doc in docs]

note_df = pd.DataFrame(note_records)

In [3]:
from datetime import datetime, timezone
import re


# Function to sanitize document IDs
def sanitize_document_id(doc_id):
    return re.sub(r'[^a-zA-Z0-9_-]', '_', doc_id)

# Sanitize document IDs in the notes
# Function to sanitize document IDs
def sanitize_document_id(doc_id):
    try:
        return re.sub(r'[^a-zA-Z0-9_-]', '_', doc_id)
    except TypeError as e:
        raise e

# Function to convert ISO 8601 timestamp to UNIX seconds
def convert_to_unix_seconds(timestamp):
    dt = datetime.fromisoformat(timestamp.replace('Z', '+00:00'))
    return int(dt.timestamp())

# These will be at UTC midnight
def convert_date_to_epoch(date_str):
    dt = datetime.strptime(date_str, '%Y-%m-%d').replace(tzinfo=timezone.utc)
    return int(dt.timestamp())


import copy

# Clone note_records
cloned_note_records = copy.deepcopy(note_records)

# Sanitize document IDs and convert timestamps in the notes
for note in cloned_note_records:
    if 'id' in note:
        if note['id'] == None:
            print("Skipping note with None ID " + str(note))
            continue
        note['originalKey'] = note['id']
        note['id'] = sanitize_document_id(note['id'])
    if 'dayAndNightOf' in note:
        note['dayAndNightOfEpoch'] = convert_date_to_epoch(note['dayAndNightOf'])
    try:
        del note['timestampWrittenUTC']
    except KeyError:
        pass
    try:
        del note['timestampForUTC']
    except KeyError:
        pass
    for key in ['timestampWritten', 'timestampFor', 'timestampUpdated']:
        if key in note:
            if type(note[key]) == str:
                note[key] = convert_to_unix_seconds(note[key])
            elif isinstance(note[key], datetime):
                note[key] = convert_to_unix_seconds(note[key].isoformat())
            elif note[key] is not None:
                print(note)
                raise ValueError(f"Unexpected type for {key}: {type(note[key])}")


    for key, value in note.items():
        if isinstance(value, datetime):
            print(note)
            raise ValueError(f"Unexpected type for {key}: {type(note[key])}")


In [4]:
cloned_note_records[0]

{'timestampWritten': 1714751288,
 'tags': ['ritalin', 'inhibition', 'autistic'],
 'note': "Interesting ritalin thing.. It suppresses my inhibition enough to make me respond quicker, and that's really making me realise that I am a bit autistic. Maybe?",
 'dayAndNightOf': '2024-05-03',
 'timestampFor': 1714751237,
 'id': '2024-05-03T16_48_08_446_01_00',
 'originalKey': '2024-05-03T16:48:08.446+01:00',
 'dayAndNightOfEpoch': 1714694400}

In [5]:
cloned_note_records[0]

{'timestampWritten': 1714751288,
 'tags': ['ritalin', 'inhibition', 'autistic'],
 'note': "Interesting ritalin thing.. It suppresses my inhibition enough to make me respond quicker, and that's really making me realise that I am a bit autistic. Maybe?",
 'dayAndNightOf': '2024-05-03',
 'timestampFor': 1714751237,
 'id': '2024-05-03T16_48_08_446_01_00',
 'originalKey': '2024-05-03T16:48:08.446+01:00',
 'dayAndNightOfEpoch': 1714694400}

In [6]:
import json

# Function to check if an object is JSON serializable
def is_json_serializable(obj):
    try:
        json.dumps(obj)
        return True
    except (TypeError, OverflowError):
        return False

# Filter cloned_note_records to find records that aren't JSON serializable
non_serializable_records = [
    note for note in cloned_note_records
    if not is_json_serializable(note)
]

# Display the filtered records
print(non_serializable_records)

[]


In [7]:
import meilisearch
import json
import json
import re
import os
import dotenv
dotenv.load_dotenv("../../.env")

client = meilisearch.Client('https://examined-life.co.uk:7700', os.getenv('MEILISEARCH_MASTER_KEY'))



In [8]:
client.index('notes').delete()
client.index('notes').add_documents(cloned_note_records)

TaskInfo(task_uid=979, index_uid='notes', status='enqueued', type='documentAdditionOrUpdate', enqueued_at=datetime.datetime(2025, 1, 26, 10, 0, 13, 390082))

In [9]:
os.getenv('MEILISEARCH_MASTER_KEY')

'30c3f8c2-c3d6-4be1-bfd3-7addafdec88b'

In [10]:
client.index('notes').update_filterable_attributes([
    'tags',
    'timestampFor',
    'dayAndNightOfEpoch',
    'originalKey'
])
client.index('notes').update_sortable_attributes([
    'timestampFor',
    'dayAndNightOfEpoch',
    'dayAndNightOf'
])
client.index('notes').update_ranking_rules([
    "sort",  # Set 'sort' as the first rule
    "words",
    "typo",
    "proximity",
    "attribute",
    "exactness"
])


TaskInfo(task_uid=982, index_uid='notes', status='enqueued', type='settingsUpdate', enqueued_at=datetime.datetime(2025, 1, 26, 10, 0, 14, 538610))

In [12]:
import requests
import os
import dotenv

dotenv.load_dotenv("../.env")

url = "http://examined-life.co.uk:7700/experimental-features/"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {os.getenv('MEILISEARCH_MASTER_KEY')}"
}
data = {
    "containsFilter": True
}

response = requests.patch(url, headers=headers, json=data)

print(response.status_code)
print(response.json())

ConnectionError: ('Connection aborted.', BadStatusLine('\x15\x03\x03\x00\x02\x022'))